### Dynamic Test Creation





### Lib Imports: 

In [21]:
import os
import openai


In [22]:
os.environ["OPENAI_API_KEY"] = "put you api key here"                           

Instructing GPT to Develop a Multiple-Choice Quiz 

To have GPT create a multiple-choice quiz, we must provide clear instructions, including the subject matter, the number of possible answer choices, and the total questions. Additionally, GPT should include the correct answer to facilitate automatic grading later on.

In [23]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [25]:

def generate_quiz_instructions(topic, question_count, answer_choices_count):
        instructions = (f"Generate a multiple-choice quiz on {topic} with {question_count} questions. "
        f"Provide {answer_choices_count} options for each question. "
        f"Include the correct answer for every question, starting with 'Correct Answer: '.")
        return instructions

In [26]:
generate_quiz_instructions("Python", 4, 4)

"Generate a multiple-choice quiz on Python with 4 questions. Provide 4 options for each question. Include the correct answer for every question, starting with 'Correct Answer: '."

### OpenAI API Invocation  
For standard text generation, we will utilize the text-davinci-003 model.

In [27]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=generate_quiz_instructions("Python", 4, 4),
                                            max_tokens=256,
                                            temperature=0.7)

In [28]:
response


<OpenAIObject text_completion id=cmpl-7DKWvRgt1iNpa13lxOr4m3bYC8mQX at 0x160a9ace900> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nQ1. What is the correct syntax to print a string in Python?\nA. print(\"string\")\nB. printf(\"string\")\nC. echo(\"string\")\nD. print(\"string\")\nCorrect Answer: A. print(\"string\")\n\nQ2. What is the type of the result of the following expression? 4 * (6 + 5)\nA. Integer\nB. Float\nC. String\nD. Boolean\nCorrect Answer: A. Integer\n\nQ3. What is the correct syntax to assign a value to a variable?\nA. x = 5\nB. var x = 5\nC. int x = 5\nD. x := 5\nCorrect Answer: A. x = 5\n\nQ4. What is the output of the following code?\n\nx = \"Python\"\nprint(x[2:4])\n\nA. Py\nB. th\nC. on\nD. yt\nCorrect Answer: B. th"
    }
  ],
  "created": 1683411537,
  "id": "cmpl-7DKWvRgt1iNpa13lxOr4m3bYC8mQX",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_toke

In [29]:
response["choices"][0]["text"]

'\n\nQ1. What is the correct syntax to print a string in Python?\nA. print("string")\nB. printf("string")\nC. echo("string")\nD. print("string")\nCorrect Answer: A. print("string")\n\nQ2. What is the type of the result of the following expression? 4 * (6 + 5)\nA. Integer\nB. Float\nC. String\nD. Boolean\nCorrect Answer: A. Integer\n\nQ3. What is the correct syntax to assign a value to a variable?\nA. x = 5\nB. var x = 5\nC. int x = 5\nD. x := 5\nCorrect Answer: A. x = 5\n\nQ4. What is the output of the following code?\n\nx = "Python"\nprint(x[2:4])\n\nA. Py\nB. th\nC. on\nD. yt\nCorrect Answer: B. th'

In [30]:
print(response["choices"][0]["text"])



Q1. What is the correct syntax to print a string in Python?
A. print("string")
B. printf("string")
C. echo("string")
D. print("string")
Correct Answer: A. print("string")

Q2. What is the type of the result of the following expression? 4 * (6 + 5)
A. Integer
B. Float
C. String
D. Boolean
Correct Answer: A. Integer

Q3. What is the correct syntax to assign a value to a variable?
A. x = 5
B. var x = 5
C. int x = 5
D. x := 5
Correct Answer: A. x = 5

Q4. What is the output of the following code?

x = "Python"
print(x[2:4])

A. Py
B. th
C. on
D. yt
Correct Answer: B. th


### Extracting Questions and Answers

It's essential to extract the questions and answers from the generated quiz to present them to students in an organized manner.

In [31]:
def generate_student_perspective(quiz, total_questions):
    student_perspective = {1: ""}
    question_index = 1
    for line in quiz.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_perspective[question_index] += line + "\n"
        else:
            if question_index < total_questions:
                question_index += 1
                student_perspective[question_index] = ""
    return student_perspective

In [32]:
generate_student_perspective(response["choices"][0]["text"], 4)

{1: '\n\nQ1. What is the correct syntax to print a string in Python?\nA. print("string")\nB. printf("string")\nC. echo("string")\nD. print("string")\n',
 2: '\nQ2. What is the type of the result of the following expression? 4 * (6 + 5)\nA. Integer\nB. Float\nC. String\nD. Boolean\n',
 3: '\nQ3. What is the correct syntax to assign a value to a variable?\nA. x = 5\nB. var x = 5\nC. int x = 5\nD. x := 5\n',
 4: '\nQ4. What is the output of the following code?\n\nx = "Python"\nprint(x[2:4])\n\nA. Py\nB. th\nC. on\nD. yt\n'}

In [33]:
print(generate_student_perspective(response["choices"][0]["text"], 4))

{1: '\n\nQ1. What is the correct syntax to print a string in Python?\nA. print("string")\nB. printf("string")\nC. echo("string")\nD. print("string")\n', 2: '\nQ2. What is the type of the result of the following expression? 4 * (6 + 5)\nA. Integer\nB. Float\nC. String\nD. Boolean\n', 3: '\nQ3. What is the correct syntax to assign a value to a variable?\nA. x = 5\nB. var x = 5\nC. int x = 5\nD. x := 5\n', 4: '\nQ4. What is the output of the following code?\n\nx = "Python"\nprint(x[2:4])\n\nA. Py\nB. th\nC. on\nD. yt\n'}


In [34]:
def retrieve_correct_answers(quiz, total_questions):
    correct_answers = {1: ""}
    question_index = 1
    for line in quiz.split("\n"):
        if line.startswith("Correct Answer:"):
            correct_answers[question_index] += line + "\n"

            if question_index < total_questions:
                        question_index += 1
                        correct_answers[question_index] = ""
    return correct_answers


In [35]:
retrieve_correct_answers(response["choices"][0]["text"], 4)

{1: 'Correct Answer: A. print("string")\n',
 2: 'Correct Answer: A. Integer\n',
 3: 'Correct Answer: A. x = 5\n',
 4: 'Correct Answer: B. th\n'}


### Quiz Simulation    
Utilizing the extracted questions, we can proceed with simulating the quiz experience.

In [36]:
def participate(student_perspective):
    submitted_answers = {}
    for question_num, question_content in student_perspective.items():
        print(question_content)
        response = input("Enter your answer: ")
        submitted_answers[question_num] = response
    return submitted_answers

In [37]:
student_answers = participate(generate_student_perspective(response["choices"][0]["text"], 4))



Q1. What is the correct syntax to print a string in Python?
A. print("string")
B. printf("string")
C. echo("string")
D. print("string")


Q2. What is the type of the result of the following expression? 4 * (6 + 5)
A. Integer
B. Float
C. String
D. Boolean


Q3. What is the correct syntax to assign a value to a variable?
A. x = 5
B. var x = 5
C. int x = 5
D. x := 5


Q4. What is the output of the following code?

x = "Python"
print(x[2:4])

A. Py
B. th
C. on
D. yt




### Automated Assessment
Utilizing the student's submitted answers and the known correct answers, we can now evaluate the quiz!

In [39]:
def evaluate(known_answers, submitted_answers):
    accurate_responses = 0
    for question_num, response in submitted_answers.items():
        if response.upper() == known_answers[question_num].upper()[16]:
             accurate_responses += 1
    score = 100 * accurate_responses / len(submitted_answers)
    if score < 60:
        pass_status = "Not passed!"
    else:
        pass_status = "Passed!"
    return f"{accurate_responses} out of {len(submitted_answers)} correct! You achieved: {score}% : {pass_status}"


In [40]:
evaluate(retrieve_correct_answers(response["choices"][0]["text"], 4), student_answers)

'3 out of 4 correct! You achieved: 75.0% : Passed!'